In [1]:
from predictNew import PreprocessData, Predict, ScrapeGlass, LinkedinScraper
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

/Users/ethandoyle/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
def remove_dups(train, test):
    train_df = pd.read_csv(train, index_col=0)
    test_df = pd.read_csv(test, index_col=0)
    final_jobs = test_df.copy()
    for test_job in test_df.jobs.values:
        for train_job in train_df.jobs.values:
            if test_job == train_job:
                final_jobs.drop(index=final_jobs[final_jobs.jobs == test_job].index[0], inplace=True)
    return final_jobs

In [8]:
final_jobs = remove_dups('data/GLASSTEST.csv', 'data/HOTTEST.csv')

In [12]:
final_jobs

,titles,companies,jobs
5,"Scientist, Formulations, Viral Vector Process ...","4.0 ★Juno Therapeutics– Seattle, WA, United St...","Juno is seeking an enthusiastic, self-driven i..."
6,Machine Learning Scientist,"3.9 ★Versive– Seattle, WA","The Role:\n\nAt Versive, we want to cut throug..."
7,Data Scientist I,4.5 ★PagerDuty– Seattle,"At PagerDuty, we believe that people do their ..."
8,Data Scientist,"4.5 ★H10 Capital– Seattle, WA",Job Description\nOur Telecommunications client...
9,Data Scientist,2.7 ★Rakuten– Seattle,We are looking for an experienced data scienti...
10,Siri - Data Scientist (Machine Learning & Natu...,"4.0 ★Apple– Seattle, Washington",Summary\nPlay a part in the ongoing revolution...
11,Data Engineer,"3.8 ★Amazon– Seattle, WA",Job Description\nDo you want to build a cuttin...


In [13]:
data = PreprocessData()

In [16]:
total_df = data.transform('data/GLASSTEST.csv', 'data/ETHANTEST.csv')

In [17]:
total_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,0,NaN,NaN,Data Scientist with an economics background fr...,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061251
1,2,Data Scientist,"3.6 ★Tableau Software– Seattle, Washington",**What youll be doing**The primary focus of th...,0.0,0.861950,0.000000,0.000000,0.000000,0.066963,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036914,0.066963,0.000000,0.000000
2,8,Senior Data Scientist,"5.0 ★Suplari– Seattle, WA",Our mission at Suplari is to shine the light o...,0.0,0.871546,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026564,0.000000,0.000000,0.000000
3,1,Data Scientist,"3.8 ★Amazon– Seattle, WA",Job Description\nThe Amazon Demand Forecasting...,0.0,0.899140,0.000000,0.042778,0.042778,0.000000,...,0.000000,0.042778,0.042778,0.042778,0.000000,0.000000,0.047164,0.000000,0.000000,0.000000
4,5,Data Scientist I,"3.8 ★Expedia– Bellevue, Washington",Expedia\n\nWho we are\n\neCP Expedia eCommerce...,0.0,0.902016,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.043292,0.000000,0.000000,0.043292,0.000000
5,3,"Data Scientist, Sr.","3.7 ★Seattle Children's Hospital– Seattle, WA",Join our Bioinformatics Delivery Team! Using t...,0.0,0.911966,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084110,0.000000,0.000000,0.000000
6,9,Data Scientist - Verticals,"4.2 ★Uber– Seattle, Washington, United States","At Uber, we ignite opportunity by setting the ...",0.0,0.917575,0.000000,0.000000,0.000000,0.000000,...,0.114174,0.000000,0.000000,0.000000,0.000000,0.000000,0.031470,0.000000,0.000000,0.000000
7,6,Data Scientist,"3.1 ★EagleView Technologies– Bellevue, WA",We are looking for talented Data Scientists to...,0.0,0.926140,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,10,Associate Data Scientist,"4.1 ★ExtraHop Networks, Inc.– Seattle, WA, Uni...",The ExtraHop platform is a novel approach to p...,0.0,0.932524,0.043019,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.043019,0.000000,0.000000,0.000000,0.000000,0.000000
9,7,Data Scientist,"3.6 ★TIBCO Software– Seattle, WA, US",Our Data Science team are central to our globa...,0.0,0.934063,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
